In [1]:
import itertools
import os
import pathlib
import sys
from itertools import product

import numpy as np
import pandas as pd

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False


from loading_classes import ImageSetLoader
from notebook_init_utils import bandicoot_check, init_notebook

In [15]:
root_dir, in_notebook = init_notebook()
bandicoot_path = pathlib.Path(os.path.expanduser("~/mnt/bandicoot")).resolve(
    strict=True
)
image_base_path = bandicoot_check(
    bandicoot_mount_path=bandicoot_path, root_dir=root_dir
)
patient_id_file = pathlib.Path(f"{root_dir}/data/patient_IDs.txt").resolve(strict=True)
patients = pd.read_csv(
    patient_id_file, header=None, names=["patient_id"]
).patient_id.tolist()

input_combinations_path = pathlib.Path(
    f"{root_dir}/2.segment_images/load_data/input_combinations.txt"
)
input_combinations_path.parent.mkdir(parents=True, exist_ok=True)

In [16]:
channel_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [17]:
# example image set path to get the image set loader working
image_set_path = pathlib.Path(f"{image_base_path}/data/NF0014_T1/zstack/C4-2/")
mask_set_path = pathlib.Path(
    f"{image_base_path}/data/NF0014_T1/segmentation_masks/C4-2/"
)
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    mask_set_path=mask_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)

In [18]:
output_dict = {
    "patient": [],
    "well_fov": [],
    "clip_limit": [],
    "input_subparent_name": [],
    "mask_subparent_name": [],
}

In [19]:
for patient in patients:
    # get the well_fov for each patient
    patient_well_fovs = pathlib.Path(
        f"{image_base_path}/data/{patient}/zstack_images/"
    ).glob("*")
    for well_fov in patient_well_fovs:
        well_fov = well_fov.name

        output_dict["patient"].append(patient)
        output_dict["well_fov"].append(well_fov)
        output_dict["clip_limit"].append(0.02)
        output_dict["input_subparent_name"].append("zstack_images")
        output_dict["mask_subparent_name"].append("segmentation_masks")

In [20]:
df = pd.DataFrame(output_dict)
print(f"Total combinations: {df.shape[0]}")
df.head()

Total combinations: 3025


,patient,well_fov,clip_limit,input_subparent_name,mask_subparent_name
0,NF0014_T1,E5-2,0.02,zstack_images,segmentation_masks
1,NF0014_T1,D5-2,0.02,zstack_images,segmentation_masks
2,NF0014_T1,G6-1,0.02,zstack_images,segmentation_masks
3,NF0014_T1,G8-1,0.02,zstack_images,segmentation_masks
4,NF0014_T1,C9-2,0.02,zstack_images,segmentation_masks


In [21]:
# write to a txt file with each row as a combination
# each column is a feature of the combination
df.to_csv(input_combinations_path, sep="\t", index=False)